In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyodbc

# KẾT NỐI PYTHON VỚI SQL SERVER

In [2]:
conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=DESKTOP-6J6UI2N\ANH; Database=Northwind; TRUSTED_CONNECTION=yes')

# GIỚI THIỆU DỰ ÁN

Ban quản lý của Northwind Traders – một nhà phân phối thực phẩm dành cho những người sành ăn quốc tế, đang tìm kiếm thông tin để đưa ra các quyết định chiến lược trong một số khía cạnh của doanh nghiệp. Các dự án ở đây tập trung vào:
- Đánh giá hiệu suất của nhân viên để tăng năng suất;
- Hiểu doanh số bán hàng và hiệu suất danh mục để tối ưu hóa chiến lược tiếp thị và hàng tồn kho;
- Phân tích tăng trưởng doanh số để xác định xu hướng, theo dõi tiến độ của công ty và đưa ra dự báo chính xác hơn;
- Đánh giá hành vi mua hàng của khách hàng để hướng tới khách hàng có giá trị cao với các ưu đãi khuyến mại

# LƯỢC ĐỒ CƠ SỞ DỮ LIỆU

![](Northwind_relationship.png)

# PHÂN TÍCH DỮ LIỆU

## Đánh giá toàn diện hiệu suất bán hàng của công ty từ góc độ nhân viên:

Mục tiêu có 2 mặt:
- Đầu tiên, đội ngũ quản lý muốn ghi nhận và khen thưởng những nhân viên có thành tích xuất sắc, thúc đẩy văn hóa xuất sắc trong tổ chức. 
- Thứ hai, họ muốn xác định những nhân viên có thể đang gặp khó khăn để có thể cung cấp các khóa đào tạo hoặc nguồn lực cần thiết nhằm giúp họ cải thiện.

Đội ngũ quản lý quan tâm đến việc khuyến khích sự cạnh tranh lành mạnh và khen thưởng những người có thành tích xuất sắc. Để làm được điều này cần phải xếp hạng nhân viên dựa trên tổng doanh số bán hàng của họ.

In [3]:
query = '''
WITH employee_sales AS
(
  SELECT e.employee_id,
         e.first_name, 
         e.last_name, 
         ROUND(SUM(unit_price * quantity * (1 - discount)),2) as total_sales
    FROM order_details AS od
    JOIN orders AS o ON od.order_id = o.order_id
    JOIN employees AS e ON o.employee_id = e.employee_id
GROUP BY e.employee_id,e.first_name,e.last_name
)
SELECT employee_id, first_name, last_name,
       RANK() OVER(ORDER BY total_sales DESC) AS Sales_rank
  FROM employee_sales;
'''

In [4]:
employee_sales = pd.read_sql_query(query,conn)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10696\61277300.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employee_sales = pd.read_sql_query(query,conn)


In [5]:
employee_sales

,employee_id,first_name,last_name,Sales_rank
0,4,Margaret,Peacock,1
1,3,Janet,Leverling,2
2,1,Nancy,Davolio,3
3,2,Andrew,Fuller,4
4,8,Laura,Callahan,5
5,7,Robert,King,6
6,9,Anne,Dodsworth,7
7,6,Michael,Suyama,8
8,5,Steven,Buchanan,9


=> Chúng ta có thể thấy Margaret Peacock và Janet Leverling là 2 nhân viên có doanh số bán hàng cao nhất. Bên cạnh đó, Michael Suyama và Steven Buchanan có doanh số bán hàng thấp nhất.

## TÍNH TỔNG DOANH THU HẰNG THÁNG

- Tiếp đến chúng ta cần quan tâm hiệu suất bán hàng tổng thể của công ty theo thời gian. Ban quản lý công ty đang tìm cách hình dung tiến trình bán hàng và xác định các xu hướng có thể định hình chiến lược tương lai của công ty.
- Việc cần làm đầu tiên là trực quan hóa tiến độ bán hàng của công ty theo thời gian hằng tháng để có thể thấy rõ xu hướng bán hàng và giúp xác định các giai đoạn hoạt động bán hàng cao hay thấp. Điều này sẽ liên quan đến việc tổng hợp dữ liệu bán hàng hàng tháng và tính toán tổng doanh thu theo tháng


In [6]:
query_1 = '''
WITH Monthly_sales AS
(
SELECT DATEADD(MONTH,DATEDIFF(MONTH,0,Order_date),0) AS month,
       ROUND(SUM(Unit_price * quantity * (1 - discount)),2) AS total_sales
FROM orders AS o 
JOIN order_details AS od ON od.order_id = o.order_id
GROUP BY DATEADD(MONTH,DATEDIFF(MONTH,0,Order_date),0)
)
SELECT 
CONCAT(SUBSTRING(CONVERT(varchar(15),month,112),1,4),'-' ,(SUBSTRING(CONVERT(varchar(15),month,112),5,2)),'-' ,(SUBSTRING(CONVERT(varchar(15),month,112),7,2))) AS Month,
       SUM(total_sales) OVER(ORDER BY month) AS running_total
FROM Monthly_sales;
'''

In [7]:
monthly_sales = pd.read_sql_query(query_1,conn)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10696\3419218183.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  monthly_sales = pd.read_sql_query(query_1,conn)


In [8]:
monthly_sales

,Month,running_total
0,1996-07-01,27861.90
1,1996-08-01,53347.18
2,1996-09-01,79728.58
3,1996-10-01,117244.30
4,1996-11-01,162844.35
5,1996-12-01,208083.98
6,1997-01-01,269342.05
7,1997-02-01,307825.68
8,1997-03-01,346372.90
9,1997-04-01,399405.85


Truy vấn trên trả về tổng doanh thu hàng tháng của cửa hàng từ tháng 7/1996 đến 5/1998. Từ bảng trên ta có thể thấy tổng doanh thu hàng tháng tăng lên 6 lần trong giai đoạn trên từ 27862 USD (7/1996) lên 1265793 USD (5/1998).

## TÍNH TĂNG TRƯỞNG DOANH SỐ HÀNG THÁNG

- Sau khi tìm hiểu tổng doanh số bán hàng theo tháng, chúng ta sẽ đi phân tích sâu hơn những số liệu này: phân tích tốc độ tăng trưởng doanh số hàng tháng. Hiểu được tốc độ tăng trưởng doanh số hàng tháng tăng hay giảm sẽ giúp chúng ta xác định được các xu hướng quan trọng.
- Để xác định tăng trưởng doanh số hàng tháng, chúng ta cần tính toán phần trăm thay đổi về doanh số bán hàng từ tháng này sang tháng khác. Điều này có thể cung cấp một bức tranh rõ ràng về các giai đoạn tăng trưởng hoặc thu hẹp, từ đó cung cấp thông tin cho chiến lược kinh doanh và đưa ra quyết định chính xác.


In [9]:
query_2 = '''
WITH 
sales_by_month AS
(
SELECT YEAR(order_date) AS Year,
       Month(order_date) AS Month,
       SUM(Unit_price * quantity *(1 - discount)) AS total_sales
FROM order_details AS od 
JOIN orders AS o ON od.order_id = o.order_id
GROUP BY YEAR(order_date),Month(order_date)
),
Sales_growth AS
(
SELECT Year, Month, total_sales,
       LAG(total_sales) OVER( ORDER BY year, month) AS Previous_Month_sales
FROM sales_by_month
)
SELECT Year, Month, total_sales, previous_month_sales,
       ROUND(((total_sales - Previous_Month_sales)/Previous_Month_sales)*100,2) AS Growth
FROM Sales_growth;
'''

In [10]:
sales_by_month = pd.read_sql_query(query_2,conn)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10696\3490299655.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_by_month = pd.read_sql_query(query_2,conn)


In [11]:
sales_by_month

,Year,Month,total_sales,previous_month_sales,Growth
0,1996,7,27861.895129,NaN,NaN
1,1996,8,25485.275067,27861.895129,-8.53
2,1996,9,26381.400128,25485.275067,3.52
3,1996,10,37515.724943,26381.400128,42.21
4,1996,11,45600.045207,37515.724943,21.55
5,1996,12,45239.630506,45600.045207,-0.79
6,1997,1,61258.070163,45239.630506,35.41
7,1997,2,38483.634949,61258.070163,-37.18
8,1997,3,38547.220106,38483.634949,0.17
9,1997,4,53032.952387,38547.220106,37.58


Nhìn kết quả của truy vấn trên, ta xác định được tốc độ tăng trưởng doanh số hàng tháng của cửa hàng. Tốc độ tăng trưởng cao nhất vào tháng 12/1997 (64.01%) và thấp nhất vào tháng 5/1998 (-85.19%). 

## XÁC ĐỊNH KHÁCH HÀNG CÓ GIÁ TRỊ CAO

- Một yếu tố không kém phần quan trọng của hoạt động kinh doanh là khách hàng. Chúng ta cần xác định những khách hàng có giá trị cao để từ đó có thể cung cấp các chương trình khuyến mãi, ưu đãi đặc biệt. Điều này có thể thúc đẩy doanh số bán hàng tăng lên, cải thiện khả năng giữ chân khách hàng và thu hút khách hàng mới.
- Để xác định khách hàng nào mang lại giá trị cao cho cửa hàng, chúng ta có thể tìm kiếm những khách hàng có giá trị đơn hàng trên mức trung bình. Những khách hàng này có thể là doanh nghiệp mua số lượng lớn hoặc cá nhân mua sản phẩm cao cấp
- Dưới đây là 10 khách hàng mang lại giá trị cao cho cửa hàng:


In [12]:
query_3 = '''
WITH order_value AS
(
SELECT o.customer_id,
       o.order_id,
       SUM(quantity * unit_price * (1-discount)) AS total_sales
FROM orders AS o
JOIN order_details AS od ON od.order_id = o.order_id
GROUP BY o.customer_id, o.order_id
)
SELECT TOP 10 customer_id, order_id,total_sales,
       CASE 
       WHEN total_sales > AVG(total_sales) OVER() THEN 'Above Average'
       ELSE 'Below Average'
       END AS 'Value Category'
FROM order_value;
'''

In [13]:
order_value = pd.read_sql_query(query_3,conn)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10696\2013254481.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  order_value = pd.read_sql_query(query_3,conn)


In [14]:
order_value

,customer_id,order_id,total_sales,Value Category
0,VINET,10248,439.999998,Below Average
1,TOMSP,10249,1863.400064,Above Average
2,HANAR,10250,1552.600022,Above Average
3,VICTE,10251,654.059985,Below Average
4,SUPRD,10252,3597.900148,Above Average
5,HANAR,10253,1444.799983,Below Average
6,CHOPS,10254,556.620010,Below Average
7,RICSU,10255,2490.499977,Above Average
8,WELLI,10256,517.800007,Below Average
9,HILAA,10257,1119.899954,Below Average


## PHẦN TRĂM DOANH THU CHO TỪNG DANH MỤC SẢN PHẨM

- Để tìm hiểu họat động của sản phẩm và danh mục sản phẩm, chúng ta cần tìm hiểu về cơ cấu doanh số bán hàng của các danh mục sản phẩm khác nhau. Bằng cách biết tỷ lệ phần trăm trong tổng doanh số của từng danh mục sản phẩm, chúng ta có thể hiểu rõ hơn về danh mục nào thúc đẩy phần lớn doanh số bán hàng của công ty.

=> Từ đó chúng ta sẽ có những quyết định đúng đắn về hàng tồn kho (VD: danh mục nào nên được dự trữ nhiều hơn) và chiến lược tiếp thị phù hợp (VD: danh mục nào nên được quảng cáo tích cực hơn)

In [15]:
query_4 = '''
WITH category_sales AS
(
SELECT c.category_id, c.category_name,
       SUM(od.unit_price * quantity * (1-discount)) AS total_sales
FROM order_details AS od 
JOIN products AS p ON p.product_id = od.product_id
JOIN categories AS c ON c.category_id = p.category_id
GROUP BY c.category_id, c.category_name
)
SELECT category_id, category_name, 
       total_sales/SUM(total_sales) OVER() * 100 AS sales_percentage
FROM category_sales
ORDER BY sales_percentage DESC;
'''

In [18]:
sales_percentage = pd.read_sql_query(query_4,conn)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10696\4096948888.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_percentage = pd.read_sql_query(query_4,conn)


In [19]:
sales_percentage

,category_id,category_name,sales_percentage
0,1,Beverages,21.162083
1,4,Dairy Products,18.526511
2,3,Confections,13.221532
3,6,Meat/Poultry,12.879069
4,8,Seafood,10.369921
5,2,Condiments,8.377917
6,7,Produce,7.898967
7,5,Grains/Cereals,7.564000


Beverages(đồ uống) là danh mục dẫn đầu về tỷ lệ phần trăm theo doanh thu với 21,16%; tiếp đến là các sản phẩm sữa (Dairy Products) với 18,53%. Produce và Grains/Cereals (Ngũ cốc) là những danh mục có tỷ lệ doanh thu nhỏ nhất lần lượt là 7,9% và 7,56%.

## TOP 3 SẢN PHẨM HÀNG ĐẦU THEO DANH MỤC

Chúng tôi muốn tìm ra 3 mặt hàng được bán ra nhiều nhất trong mỗi loại sản phẩm. Điều này sẽ cho ta xác định được những sản phẩm xuất sắc và đảm bảo rằng những sản phẩm này được lưu giữ trong kho và được tiếp thị một cách nổi bật.

In [20]:
query_5 = '''
WITH Product_sales AS
(
SELECT p.category_id,p.product_id,p.product_name,
       SUM(od.unit_price * od.quantity*(1-discount)) AS total_sales
FROM products AS p
JOIN order_details AS od ON od.product_id = p.product_id
GROUP BY p.category_id,p.product_id,p.product_name
)
SELECT category_id, product_id,product_name,total_sales, rank
FROM (
SELECT Category_id, product_id,product_name,total_sales,
       ROW_NUMBER() OVER(PARTITION BY category_id ORDER BY total_sales DESC) AS rank
FROM Product_sales) AS top_3
WHERE rank <=3;

'''

In [21]:
rank = pd.read_sql_query(query_5,conn)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10696\433441652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rank = pd.read_sql_query(query_5,conn)


In [22]:
rank

,category_id,product_id,product_name,total_sales,rank
0,1,38,Côte de Blaye,141396.735619,1
1,1,43,Ipoh Coffee,23526.699836,2
2,1,2,Chang,16355.959902,3
3,2,63,Vegie-spread,16701.095046,1
4,2,61,Sirop d'érable,14352.599870,2
5,2,65,Louisiana Fiery Hot Pepper Sauce,13869.889422,3
6,3,62,Tarte au sucre,47234.969946,1
7,3,20,Sir Rodney's Marmalade,22563.360301,2
8,3,26,Gumbär Gummibärchen,19849.144148,3
9,4,59,Raclette Courdavault,71155.699911,1
